In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import load_model

# Custom objects required for loading CheXpert multi-label classification models
import sys 
sys.path.append('..')
from loss import (
    set_binary_crossentropy_weighted_loss
)

2021-11-27 19:23:36.156094: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


### 1- Creating ensemble model from the trained multi-label classification models

In [2]:
densenet201_model = load_model(
    "../trained_models/densenet201.h5",
    custom_objects={
        "binary_crossentropy_weighted_loss": set_binary_crossentropy_weighted_loss
    }
)

inceptionresnetv2_model = load_model(
    "../trained_models/inceptionresnetv2.h5",
    custom_objects={
        "binary_crossentropy_weighted_loss": set_binary_crossentropy_weighted_loss
    }
)

2021-11-27 19:23:40.399925: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-11-27 19:23:40.412174: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-11-27 19:23:40.498183: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-27 19:23:40.500053: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA TITAN RTX computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.65GiB deviceMemoryBandwidth: 625.94GiB/s
2021-11-27 19:23:40.500109: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-11-27 19:23:40.555994: I tensorflow/stream_executor/platform/default/d

In [3]:
# Combining all three models to one ensemble tensorflow.keras models
#  renaming the model names is required for this step

# Stackoverflow reference: https://stackoverflow.com/a/57794744/10086080
densenet201_model._name = "DenseNet-201"
inceptionresnetv2_model._name = "Inception-ResNet-V2"

In [4]:
# Taken from: https://stackoverflow.com/a/67648035/10086080
models = [densenet201_model, inceptionresnetv2_model]

model_input = tf.keras.Input(shape=(512, 512, 1))
model_outputs = [model(model_input) for model in models]

ensemble_output = tf.keras.layers.Average()(model_outputs)
ensemble_model = tf.keras.Model(inputs=model_input, outputs=ensemble_output)

In [5]:
ensemble_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 1) 0                                            
__________________________________________________________________________________________________
DenseNet-201 (Functional)       (None, 14)           18342606    input_1[0][0]                    
__________________________________________________________________________________________________
Inception-ResNet-V2 (Functional (None, 14)           54357678    input_1[0][0]                    
__________________________________________________________________________________________________
average (Average)               (None, 14)           0           DenseNet-201[0][0]               
                                                                 Inception-ResNet-V2[0][0]    

### 2- Saving ensemble model

In [6]:
ensemble_model.save("../trained_models/ensemble-model.h5")

### 3- Testing loading the saved ensemble model

In [7]:
ensemble_model = load_model(
    "../trained_models/ensemble-model.h5",
    compile=False,
    custom_objects={
        "binary_crossentropy_weighted_loss": set_binary_crossentropy_weighted_loss
    }
)

In [8]:
ensemble_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 1) 0                                            
__________________________________________________________________________________________________
DenseNet-201 (Functional)       (None, 14)           18342606    input_1[0][0]                    
__________________________________________________________________________________________________
Inception-ResNet-V2 (Functional (None, 14)           54357678    input_1[0][0]                    
__________________________________________________________________________________________________
average (Average)               (None, 14)           0           DenseNet-201[0][0]               
                                                                 Inception-ResNet-V2[0][0]    